In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as sta
from scipy.stats import norm
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [33]:
#--train and test data--
train_df = pd.read_csv('D:/DataAnalysis/2019未来高校AI挑战赛_城市-房产租金预测/train_data.csv')
test_df = pd.read_csv('D:/DataAnalysis/2019未来高校AI挑战赛_城市-房产租金预测/test_a.csv')

In [34]:
print(train_df.shape)
print(test_df.shape)

(41440, 51)
(2469, 50)


In [35]:
def proprecessing(train_df,test_df):
    
    ## fill with null values
    fill_na = ["uv","pv"]
    for fill in fill_na:
        train_df[fill].fillna(train_df[fill].mean(),inplace=True)
        test_df[fill].fillna(test_df[fill].mean(),inplace=True)
        train_df[fill] = train_df[fill].astype(int)
        test_df[fill] = test_df[fill].astype(int)
        
    ## 使用孤立森林来检测tradeMoney的异常值点
    from sklearn.ensemble import IsolationForest
    IForest = IsolationForest(contamination=0.02)
    IForest.fit(train_df['tradeMoney'].values.reshape(-1,1))
    ypred = IForest.predict(train_df['tradeMoney'].values.reshape(-1,1))
    drop_index = train_df[ypred==-1].index
    train_df.drop(drop_index,axis=0,inplace=True)
    
    ##--area--
    train_df = train_df[(train_df['area']>=10)&(train_df['area']<=200)]
    
    ##组合特征进行筛选
    #list(train_df[['region','tradeMoney','area']].groupby('region'))
    train_df.drop(train_df[(train_df['region']=='RG00001')&(train_df['area']>=100)&
                       (train_df['tradeMoney']<=2000)].index,inplace=True)
    # train_df.drop(train_df[(train_df['region']=='RG00001')&(train_df['area']<=10)&
    #                     (train_df['tradeMoney']>=3000)].index,inplace=True)
    train_df.drop(train_df[(train_df['region']=='RG00002')&(train_df['area']>=100)&
                           (train_df['tradeMoney']<=1000)].index,inplace=True)
    # train_df.drop(train_df[(train_df['region']=='RG00003')&(train_df['area']<=10)&
    #                        (train_df['tradeMoney']>=4000)].index,inplace=True)
    train_df.drop(train_df[(train_df['region']=='RG00003')&(train_df['area']>=150)&
                           (train_df['tradeMoney']<=2000)].index,inplace=True)
    train_df.drop(train_df[(train_df['region']=='RG00004')&(train_df['area']>=100)&
                           (train_df['tradeMoney']<=1000)].index,inplace=True)
    train_df.drop(train_df[(train_df['region']=='RG00005')&(train_df['area']>=150)&
                           (train_df['tradeMoney']<=2000)].index,inplace=True)
    train_df.drop(train_df[(train_df['region']=='RG00006')&(train_df['area']>=100)&
                           (train_df['tradeMoney']<1000)].index,inplace=True)
    train_df.drop(train_df[(train_df['region']=='RG00008')&(train_df['area']>100)&
                           (train_df['tradeMoney']<=2000)].index,inplace=True)
    train_df.drop(train_df[(train_df['region']=='RG00009')&(train_df['area']>=100)&
                           (train_df['tradeMoney']<=1000)].index,inplace=True)
    # train_df.drop(train_df[(train_df['region']=='RG00010')&(train_df['area']>=100)&
    #                        (train_df['tradeMoney']<=2000)].index,inplace=True)
    train_df.drop(train_df[(train_df['region']=='RG00011')&(train_df['area']>=150)&
                           (train_df['tradeMoney']<=2000)].index,inplace=True)
    
    data_df = pd.concat([train_df,test_df])
    data_df.drop(['city','ID','tradeMoney'],axis=1,inplace=True)
    
    #--rentType--
    data_df['rentType'].replace('--','未知方式',inplace=True)

    #--buildYear--
    median_build = data_df.loc[data_df['buildYear']!='暂无信息','buildYear'].median()
    data_df['buildYear'].replace("暂无信息",median_build,inplace=True)
    data_df['buildYear'] = data_df["buildYear"].astype(int)
    #data_df["buildYear"] = 2019-data_df["buildYear"]

    #--houseType--
    data_df['rooms'] = data_df['houseType'].apply(lambda x:x.split('室')[0][-1]).astype(int)
    data_df['halls'] = data_df['houseType'].apply(lambda x:x.split('厅')[0][-1]).astype(int)
    data_df['bathrooms'] = data_df['houseType'].apply(lambda x:x.split('卫')[0][-1]).astype(int)
    data_df['rooms_bathrooms'] = (data_df['bathrooms']+1)/(data_df['rooms']+1)
    data_df['rooms_bathrooms'] = data_df['rooms_bathrooms'].astype(int)
    data_df.drop('houseType',axis=1,inplace=True)
    
    #--tradeTime--
    data_df['Month'] = data_df['tradeTime'].apply(lambda x:x.split('/')[1]).astype(int)
    data_df['Date'] = data_df['tradeTime'].apply(lambda x:x.split('/')[2]).astype(int) 
    data_df.drop('tradeTime',axis=1,inplace=True)
    
    return train_df,data_df

In [36]:
train_df,data_df = proprecessing(train_df,test_df)

In [37]:
def Feature_Engineering(data_df):
    
    numerical_col = [col for col in data_df.columns if data_df[col].dtypes!='object']
    
    ##--Supporting facilities--
    data_df['All_School'] = 2*data_df['interSchoolNum']/data_df['interSchoolNum'].mean() + (
        data_df['privateSchoolNum']/data_df['privateSchoolNum'].mean()+
        data_df['schoolNum']/data_df['schoolNum'].mean())
    data_df['Transportation'] = 5*data_df['subwayStationNum']/data_df['subwayStationNum'].mean() + data_df['busStationNum']/data_df['busStationNum'].mean()
    data_df['All_hospital'] = 3*data_df['hospitalNum']/data_df['hospitalNum'].mean()+data_df['drugStoreNum']/data_df['drugStoreNum'].mean()
    data_df['All_mall'] = data_df['mallNum']/data_df['mallNum'].mean()+data_df['superMarketNum']/data_df['superMarketNum'].mean()
    data_df['other_num'] = data_df['bankNum']/data_df['bankNum'].mean() + (
        data_df['gymNum']/data_df['gymNum'].mean()+data_df['parkNum']/data_df['parkNum'].mean()
        +data_df['shopNum']/data_df['shopNum'].mean())
    data_df.drop(['subwayStationNum', 'busStationNum',
                   'interSchoolNum', 'schoolNum', 'privateSchoolNum',
                   'hospitalNum', 'drugStoreNum', 'mallNum', 'superMarketNum', 
                  'gymNum', 'bankNum', 'shopNum', 'parkNum'],axis=1, inplace=True)
    
    #--Land--
    data_df['SecMeanArea'] = data_df['totalTradeArea']/data_df['tradeSecNum']
    #data_df['SecMeanArea'].replace(np.inf,0,inplace=True)
    data_df['SecMeanArea'].fillna(0,inplace=True)
    data_df['trade_supplyNew_rate'] = data_df['tradeNewNum']/data_df['supplyNewNum']
    #data_df['trade_supplyNew_rate'].replace(np.inf,0,inplace=True)
    data_df['trade_supplyNew_rate'].fillna(0,inplace=True)
    data_df['remain_supply_rate'] = data_df['remainNewNum']/data_df['supplyNewNum']
    #data_df['remain_supply_rate'].replace(np.inf,0,inplace=True)
    data_df['remain_supply_rate'].fillna(0,inplace=True)
    data_df['trade_supplyLand_rate'] = data_df['tradeLandArea']/data_df['supplyLandArea']
    data_df['trade_supplyLand_rate'].fillna(0,inplace=True)
    
    ##--group by--
    inter = data_df.groupby('communityName')['area'].agg({'com_area_mean':'mean','com_area_std':'std'})
    inter.fillna(0,inplace=True)
    data_df = data_df.merge(inter,on='communityName',how='left')
    
    inter = data_df.groupby('plate')['area'].agg({'pla_are_mean':'mean','pla_are_std':'std'})
    inter.fillna(0,inplace=True)
    data_df = data_df.merge(inter,on='plate',how='left')
    
    
    #--buildYear--
    inter = data_df.groupby('plate')['buildYear'].agg({'pla_build_mean':'mean','pla_build_std':'std'})
    inter.fillna(0,inplace=True)
    data_df = data_df.merge(inter,on='plate',how='left')
    data_df.pla_build_mean = data_df.pla_build_mean.astype(int)
    data_df['pla_build_diff'] = data_df.buildYear - data_df.pla_build_mean
    
    #--school--
    data_df['plate_school'] = data_df.groupby('plate')['All_School'].transform('sum')
    data_df['commit_plate_school'] = data_df.groupby(['communityName','plate'])['All_School'].transform('sum')
    data_df['school_ratio'] = data_df['commit_plate_school']/data_df['plate_school']
    data_df['school_ratio'].replace(np.inf,-1,inplace=True)
    
    #--Transportation--
    data_df['plate_Transp'] = data_df.groupby('plate')['Transportation'].transform('sum')
    data_df['commit_plate_Transp'] = data_df.groupby(['communityName','plate'])['Transportation'].transform('sum')
    data_df['Transportation_ratio'] = data_df['commit_plate_Transp']/data_df['plate_Transp']
    data_df['Transportation_ratio'].replace(np.inf,-1,inplace=True)
    
    #--hospital--
    data_df['plate_hospital'] = data_df.groupby('plate')['All_hospital'].transform('sum')
    data_df['commit_plate_hospital'] = data_df.groupby(['communityName','plate'])['All_hospital'].transform('sum')
    data_df['School_ratio'] = data_df['commit_plate_hospital']/data_df['plate_hospital']
    data_df['School_ratio'].replace(np.inf,-1,inplace=True)
    
    #--mall--
    data_df['plate_mall'] = data_df.groupby('plate')['All_mall'].transform('sum')
    data_df['commit_plate_mall'] = data_df.groupby(['communityName','plate'])['All_mall'].transform('sum')
    data_df['mall_ratio'] = data_df['commit_plate_mall']/data_df['plate_mall']
    data_df['mall_ratio'].replace(np.inf,-1,inplace=True)
    
    #--other_num--
    data_df['plate_other_num'] = data_df.groupby('plate')['other_num'].transform('sum')
    data_df['commit_plate_other'] = data_df.groupby(['communityName','plate'])['other_num'].transform('sum')
    data_df['other_ratio'] = data_df['commit_plate_other']/data_df['plate_other_num']
    data_df['other_ratio'].replace(np.inf,-1,inplace=True)
    data_df.drop(['pla_build_mean','plate_school','commit_plate_school','plate_Transp','commit_plate_Transp',
                  'plate_hospital','commit_plate_hospital','plate_mall','commit_plate_mall','plate_other_num',
                 'commit_plate_other'],axis=1,inplace=True)
    
    ##--Cluster聚类分析--
    from sklearn.preprocessing import LabelEncoder
    Label_list = ['plate','region','houseToward','communityName','houseDecoration','rentType','houseFloor']
    for col_4 in Label_list:
        data_df[col_4] = data_df[col_4].astype(str)
        data_df[col_4] = LabelEncoder().fit_transform(data_df[col_4])

    # Onehot_list = ['houseDecoration','rentType','houseFloor']
    # for col_5 in Onehot_list:
    #     data_df[col_5] = data_df[col_5].astype(str)
    #     inter_col = pd.get_dummies(data_df[col_5])
    #     data_df.drop(col_5,axis=1,inplace=True)
    #     data_df = pd.concat([data_df,inter_col],axis=1)
    from sklearn.mixture import GaussianMixture
    gmm = GaussianMixture(n_components=5,covariance_type='full',random_state=2019)
    cluster_col = ['communityName','buildYear','plate','region','landMeanPrice','lookNum','newWorkers',
                   'residentPopulation','totalFloor','houseDecoration','tradeMeanPrice','tradeSecNum',
                  'totalNewTradeMoney','totalNewTradeArea','tradeNewMeanPrice','tradeNewNum','remainNewNum',
                  'landTotalPrice','landMeanPrice','totalWorkers','All_School','Transportation','All_hospital',
                  'All_mall','other_num']
    data_df['cluster'] = gmm.fit_predict(data_df[cluster_col])
    
    cluster_col_2 = ['totalFloor','houseDecoration','communityName','region','plate','buildYear','All_School',
                 'Transportation','All_hospital','All_mall','other_num']
    for i in range(len(cluster_col_2)):
        for j in range(i+1,len(cluster_col_2)):
            data_df[cluster_col_2[i]+'_'+cluster_col_2[j]] = data_df.groupby(['cluster',
                                                                              cluster_col_2[i]])[cluster_col_2[j]].transform('mean')
            data_df[cluster_col_2[i]+'_'+cluster_col_2[j]].fillna(0,inplace=True)
    return data_df

In [38]:
data_df = Feature_Engineering(data_df)

In [53]:
dataNa = data_df.isna().sum()
dataNa[dataNa>0]

Series([], dtype: int64)

## Model Selection

In [40]:
from sklearn.preprocessing import LabelEncoder

In [41]:
categorical_new = [col for col in data_df.columns if data_df[col].dtypes!='object']
data_df[categorical_new].nunique().sort_values(ascending=False)

area                              9752
communityName                     4033
com_area_mean                     3726
com_area_std                      3104
Transportation_ratio              1327
mall_ratio                        1327
other_ratio                       1326
School_ratio                      1326
school_ratio                      1324
pv                                 714
totalTradeArea                     707
tradeMeanPrice                     707
SecMeanArea                        707
totalTradeMoney                    706
uv                                 653
totalNewTradeMoney                 561
tradeNewMeanPrice                  560
totalNewTradeArea                  535
remainNewNum                       393
communityName_All_School           362
communityName_Transportation       352
communityName_other_num            336
tradeSecNum                        334
communityName_All_mall             328
communityName_All_hospital         324
buildYear_other_num      

In [42]:
print(data_df.shape)
print(train_df.shape)
print(test_df.shape)

(41640, 115)
(39171, 51)
(2469, 50)


In [43]:
Y = train_df['tradeMoney']
m = len(Y)
#data_df.drop('tradeMoney',axis=1)
X = data_df.iloc[:m,:]
X_test = data_df.iloc[m:,:]

In [44]:
print(X.shape)
print(X_test.shape)

(39171, 115)
(2469, 115)


In [45]:
def online_score(pred):
    print("预测结果最大值：{},预测结果最小值：{}".format(pred.max(),pred.min()))
    # a榜测分
    conmbine1 = pd.read_csv("D:/DataAnalysis/2019未来高校AI挑战赛_城市-房产租金预测/sub_a_913.csv",engine = "python",header=None)
    score1 = r2_score(pred, conmbine1.values)
    print("对比913分数:{}".format(score1))

### Lightgbm

In [46]:
from sklearn.metrics import r2_score
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold,cross_val_score
import time

In [47]:
params = {
    'num_leaves': 31,
    'min_data_in_leaf': 20,
    'min_child_samples':20,
    'objective': 'regression',
    'learning_rate': 0.01,
    "boosting": "gbdt",
    "feature_fraction": 0.8,
    "bagging_freq": 1,
    "bagging_fraction": 0.85,
    "bagging_seed": 23,
    "metric": 'rmse',
    "lambda_l1": 0.2,
    "nthread": 4,
}

In [48]:
folds = KFold(n_splits=5,shuffle=True,random_state=2019)
oof_lgb = np.zeros(len(X))
pred_lgb = np.zeros(len(X_test))
start = time.time()
for fold_,(trn_idx,val_idx) in enumerate(folds.split(X.values,Y.values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(X.iloc[trn_idx],label=Y.iloc[trn_idx])
    
    val_data = lgb.Dataset(X.iloc[val_idx],label=Y.iloc[val_idx])
    

    num_round = 10000
    clf = lgb.train(params,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=500,
                    early_stopping_rounds = 200)
        
    oof_lgb[val_idx] = clf.predict(X.iloc[val_idx], num_iteration=clf.best_iteration)
    pred_lgb += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits

print("CV Score: {:<8.5f}".format(r2_score( Y, oof_lgb)))

fold n°0
Training until validation scores don't improve for 200 rounds.
[500]	training's rmse: 827.17	valid_1's rmse: 893.323
[1000]	training's rmse: 731.743	valid_1's rmse: 844.13
[1500]	training's rmse: 675.872	valid_1's rmse: 822.906
[2000]	training's rmse: 635.172	valid_1's rmse: 810.632
[2500]	training's rmse: 603.61	valid_1's rmse: 802.057
[3000]	training's rmse: 577.019	valid_1's rmse: 796.433
[3500]	training's rmse: 554.415	valid_1's rmse: 792.383
[4000]	training's rmse: 534.156	valid_1's rmse: 789.392
[4500]	training's rmse: 515.822	valid_1's rmse: 787.073
[5000]	training's rmse: 499.308	valid_1's rmse: 785.683
[5500]	training's rmse: 484.286	valid_1's rmse: 784.338
[6000]	training's rmse: 470.482	valid_1's rmse: 783.273
[6500]	training's rmse: 457.754	valid_1's rmse: 782.755
[7000]	training's rmse: 445.738	valid_1's rmse: 782.269
[7500]	training's rmse: 434.578	valid_1's rmse: 781.72
Early stopping, best iteration is:
[7543]	training's rmse: 433.623	valid_1's rmse: 781.693
fo

In [18]:
online_score(pred_lgb)

预测结果最大值：15586.884674005933,预测结果最小值：1112.3936669063282
对比913分数:0.9803110140086625


### Xgboost

In [57]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [58]:
# xgb = XGBClassifier(n_estimators=1000,
#                     learning_rate=0.05,
#                     max_depth=5,
#                     min_child_weight=2,
#                     gamma=0,
#                     subsample=0.8,
#                     colsample_bytree=0.8,
#                     nthread=3,
#                     seed=2019)

In [59]:
params = {'learning_rate':0.1,
          'n_estimators':500,
          'max_depth':5,
          'min_child_weight':1,
          'gamma':0,
          'subsample':0.8,
          'colsample_bytree':0.8,
          'nthread':3}

In [ ]:
##--learning_rate--
clf = XGBClassifier(**params)
grid_params = {'learning_rate':np.linspace(0.01,0.2,20)
              }
grid = GridSearchCV(clf,grid_params)
grid.fit(X,Y)
print(grid.best_params_)

In [ ]:
##--n_estimators--
grid_params = {'n_estimators':list(range(500,2000,100))
              }
grid = GridSearchCV(clf,grid_params)
grid.fit(X,Y)
print(grid.best_params_)

In [ ]:
##--max_depth/min_child_weight--
grid_params = {'max_depth':list(range(3,15,1)),
               'min_child_weight':list(range(1,6,2))
              }
grid = GridSearchCV(clf,grid_params)
grid.fit(X,Y)
print(grid.best_params_)

In [ ]:
##--gamma--
grid_params = {'gamma':[i/10.0 for i in range(0,5)],
              'subsample':[i/10.0 for i in range(6,10)]
              }
grid = GridSearchCV(clf,grid_params)
grid.fit(X,Y)
print(grid.best_params_)

In [21]:
# xgb.fit(X,Y)
# pred_xgb = xgb.predict(X_test)

In [22]:
# online_score(pred_xgb)

预测结果最大值：17000.0,预测结果最小值：0.0
对比913分数:0.8646722047911408


### GBDT 

In [54]:
from sklearn.ensemble import GradientBoostingRegressor

In [55]:
gbdt = GradientBoostingRegressor(n_estimators=3000,
                                 learning_rate=0.04,
                                 max_depth=6,
                                 max_features='sqrt',
                                 min_samples_leaf=15,
                                 min_samples_split=10,
                                 loss='huber',
                                 random_state=2019)

In [56]:
gbdt.fit(X,Y)
pred_gbdt = gbdt.predict(X_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
online_score(pred_gbdt)

### CatBoost

In [20]:
from catboost import CatBoostRegressor

In [24]:
catb = CatBoostRegressor(iterations=5000,
                        depth=6,
                        learning_rate=0.05,
                        eval_metric='R2',
                        leaf_estimation_method = "Newton",
                        l2_leaf_reg=3)

In [25]:
catb.fit(X,Y)
pred_catb = catb.predict(X_test)

0:	learn: -2.7652331	total: 170ms	remaining: 14m 12s
1:	learn: -2.4441015	total: 322ms	remaining: 13m 23s
2:	learn: -2.1438063	total: 484ms	remaining: 13m 25s
3:	learn: -1.8785292	total: 635ms	remaining: 13m 13s
4:	learn: -1.6400569	total: 807ms	remaining: 13m 26s
5:	learn: -1.4220497	total: 960ms	remaining: 13m 18s
6:	learn: -1.2226018	total: 1.12s	remaining: 13m 18s
7:	learn: -1.0380622	total: 1.27s	remaining: 13m 14s
8:	learn: -0.8749730	total: 1.44s	remaining: 13m 17s
9:	learn: -0.7294004	total: 1.6s	remaining: 13m 18s
10:	learn: -0.5930988	total: 1.77s	remaining: 13m 21s
11:	learn: -0.4671685	total: 1.92s	remaining: 13m 17s
12:	learn: -0.3589070	total: 2.08s	remaining: 13m 18s
13:	learn: -0.2565550	total: 2.23s	remaining: 13m 13s
14:	learn: -0.1619574	total: 2.39s	remaining: 13m 15s
15:	learn: -0.0764387	total: 2.54s	remaining: 13m 12s
16:	learn: -0.0000059	total: 2.71s	remaining: 13m 14s
17:	learn: 0.0717968	total: 2.88s	remaining: 13m 17s
18:	learn: 0.1325720	total: 3.04s	remain

154:	learn: 0.8576127	total: 25.7s	remaining: 13m 23s
155:	learn: 0.8578554	total: 25.8s	remaining: 13m 22s
156:	learn: 0.8578598	total: 26s	remaining: 13m 21s
157:	learn: 0.8578639	total: 26s	remaining: 13m 17s
158:	learn: 0.8581267	total: 26.2s	remaining: 13m 17s
159:	learn: 0.8584661	total: 26.3s	remaining: 13m 16s
160:	learn: 0.8586659	total: 26.5s	remaining: 13m 16s
161:	learn: 0.8586962	total: 26.6s	remaining: 13m 15s
162:	learn: 0.8587331	total: 26.8s	remaining: 13m 15s
163:	learn: 0.8590228	total: 26.9s	remaining: 13m 14s
164:	learn: 0.8590259	total: 27.1s	remaining: 13m 12s
165:	learn: 0.8590697	total: 27.3s	remaining: 13m 13s
166:	learn: 0.8593131	total: 27.5s	remaining: 13m 14s
167:	learn: 0.8594100	total: 27.6s	remaining: 13m 15s
168:	learn: 0.8594914	total: 27.8s	remaining: 13m 15s
169:	learn: 0.8596697	total: 28s	remaining: 13m 15s
170:	learn: 0.8598262	total: 28.1s	remaining: 13m 14s
171:	learn: 0.8599221	total: 28.3s	remaining: 13m 13s
172:	learn: 0.8600149	total: 28.4s

309:	learn: 0.8765432	total: 49.3s	remaining: 12m 26s
310:	learn: 0.8765973	total: 49.4s	remaining: 12m 25s
311:	learn: 0.8767251	total: 49.6s	remaining: 12m 25s
312:	learn: 0.8768001	total: 49.8s	remaining: 12m 25s
313:	learn: 0.8769025	total: 49.9s	remaining: 12m 24s
314:	learn: 0.8769371	total: 50s	remaining: 12m 24s
315:	learn: 0.8770250	total: 50.2s	remaining: 12m 23s
316:	learn: 0.8771574	total: 50.3s	remaining: 12m 23s
317:	learn: 0.8771577	total: 50.4s	remaining: 12m 21s
318:	learn: 0.8773375	total: 50.5s	remaining: 12m 21s
319:	learn: 0.8773726	total: 50.7s	remaining: 12m 21s
320:	learn: 0.8775586	total: 50.8s	remaining: 12m 20s
321:	learn: 0.8775915	total: 51s	remaining: 12m 20s
322:	learn: 0.8777575	total: 51.1s	remaining: 12m 20s
323:	learn: 0.8779004	total: 51.3s	remaining: 12m 20s
324:	learn: 0.8780203	total: 51.4s	remaining: 12m 19s
325:	learn: 0.8780611	total: 51.6s	remaining: 12m 19s
326:	learn: 0.8781800	total: 51.7s	remaining: 12m 19s
327:	learn: 0.8782522	total: 51.

463:	learn: 0.8887876	total: 1m 12s	remaining: 11m 53s
464:	learn: 0.8888306	total: 1m 13s	remaining: 11m 53s
465:	learn: 0.8888865	total: 1m 13s	remaining: 11m 53s
466:	learn: 0.8889905	total: 1m 13s	remaining: 11m 52s
467:	learn: 0.8890333	total: 1m 13s	remaining: 11m 52s
468:	learn: 0.8890957	total: 1m 13s	remaining: 11m 52s
469:	learn: 0.8891539	total: 1m 13s	remaining: 11m 52s
470:	learn: 0.8892363	total: 1m 14s	remaining: 11m 52s
471:	learn: 0.8893298	total: 1m 14s	remaining: 11m 51s
472:	learn: 0.8893335	total: 1m 14s	remaining: 11m 51s
473:	learn: 0.8893914	total: 1m 14s	remaining: 11m 51s
474:	learn: 0.8894948	total: 1m 14s	remaining: 11m 51s
475:	learn: 0.8895921	total: 1m 14s	remaining: 11m 50s
476:	learn: 0.8895936	total: 1m 14s	remaining: 11m 50s
477:	learn: 0.8896079	total: 1m 15s	remaining: 11m 50s
478:	learn: 0.8896130	total: 1m 15s	remaining: 11m 49s
479:	learn: 0.8896163	total: 1m 15s	remaining: 11m 49s
480:	learn: 0.8896260	total: 1m 15s	remaining: 11m 49s
481:	learn

613:	learn: 0.8966142	total: 1m 34s	remaining: 11m 18s
614:	learn: 0.8966240	total: 1m 35s	remaining: 11m 17s
615:	learn: 0.8966396	total: 1m 35s	remaining: 11m 17s
616:	learn: 0.8966506	total: 1m 35s	remaining: 11m 17s
617:	learn: 0.8966534	total: 1m 35s	remaining: 11m 17s
618:	learn: 0.8967262	total: 1m 35s	remaining: 11m 17s
619:	learn: 0.8968047	total: 1m 35s	remaining: 11m 16s
620:	learn: 0.8968355	total: 1m 35s	remaining: 11m 16s
621:	learn: 0.8970005	total: 1m 36s	remaining: 11m 16s
622:	learn: 0.8970028	total: 1m 36s	remaining: 11m 16s
623:	learn: 0.8970051	total: 1m 36s	remaining: 11m 16s
624:	learn: 0.8970348	total: 1m 36s	remaining: 11m 15s
625:	learn: 0.8970358	total: 1m 36s	remaining: 11m 15s
626:	learn: 0.8970885	total: 1m 36s	remaining: 11m 15s
627:	learn: 0.8971030	total: 1m 36s	remaining: 11m 15s
628:	learn: 0.8971723	total: 1m 37s	remaining: 11m 14s
629:	learn: 0.8971805	total: 1m 37s	remaining: 11m 14s
630:	learn: 0.8971820	total: 1m 37s	remaining: 11m 14s
631:	learn

...

4383:	learn: 0.9268317	total: 10m 31s	remaining: 1m 28s
4384:	learn: 0.9268317	total: 10m 32s	remaining: 1m 28s
4385:	learn: 0.9268318	total: 10m 32s	remaining: 1m 28s
4386:	learn: 0.9268318	total: 10m 32s	remaining: 1m 28s
4387:	learn: 0.9268318	total: 10m 32s	remaining: 1m 28s
4388:	learn: 0.9268335	total: 10m 32s	remaining: 1m 28s
4389:	learn: 0.9268337	total: 10m 32s	remaining: 1m 27s
4390:	learn: 0.9268346	total: 10m 32s	remaining: 1m 27s
4391:	learn: 0.9268349	total: 10m 33s	remaining: 1m 27s
4392:	learn: 0.9268349	total: 10m 33s	remaining: 1m 27s
4393:	learn: 0.9268350	total: 10m 33s	remaining: 1m 27s
4394:	learn: 0.9268455	total: 10m 33s	remaining: 1m 27s
4395:	learn: 0.9268456	total: 10m 33s	remaining: 1m 27s
4396:	learn: 0.9268504	total: 10m 33s	remaining: 1m 26s
4397:	learn: 0.9268527	total: 10m 33s	remaining: 1m 26s
4398:	learn: 0.9268535	total: 10m 34s	remaining: 1m 26s
4399:	learn: 0.9268550	total: 10m 34s	remaining: 1m 26s
4400:	learn: 0.9268553	total: 10m 34s	remaining:

4530:	learn: 0.9270871	total: 10m 52s	remaining: 1m 7s
4531:	learn: 0.9270878	total: 10m 52s	remaining: 1m 7s
4532:	learn: 0.9270878	total: 10m 52s	remaining: 1m 7s
4533:	learn: 0.9270878	total: 10m 53s	remaining: 1m 7s
4534:	learn: 0.9270883	total: 10m 53s	remaining: 1m 6s
4535:	learn: 0.9270905	total: 10m 53s	remaining: 1m 6s
4536:	learn: 0.9270911	total: 10m 53s	remaining: 1m 6s
4537:	learn: 0.9270911	total: 10m 53s	remaining: 1m 6s
4538:	learn: 0.9270915	total: 10m 53s	remaining: 1m 6s
4539:	learn: 0.9270921	total: 10m 54s	remaining: 1m 6s
4540:	learn: 0.9270921	total: 10m 54s	remaining: 1m 6s
4541:	learn: 0.9270921	total: 10m 54s	remaining: 1m 5s
4542:	learn: 0.9270923	total: 10m 54s	remaining: 1m 5s
4543:	learn: 0.9270924	total: 10m 54s	remaining: 1m 5s
4544:	learn: 0.9270924	total: 10m 54s	remaining: 1m 5s
4545:	learn: 0.9270981	total: 10m 55s	remaining: 1m 5s
4546:	learn: 0.9270981	total: 10m 55s	remaining: 1m 5s
4547:	learn: 0.9270994	total: 10m 55s	remaining: 1m 5s
4548:	lear

4683:	learn: 0.9273784	total: 11m 15s	remaining: 45.6s
4684:	learn: 0.9273785	total: 11m 15s	remaining: 45.4s
4685:	learn: 0.9273793	total: 11m 15s	remaining: 45.3s
4686:	learn: 0.9273856	total: 11m 16s	remaining: 45.1s
4687:	learn: 0.9273859	total: 11m 16s	remaining: 45s
4688:	learn: 0.9273860	total: 11m 16s	remaining: 44.9s
4689:	learn: 0.9273912	total: 11m 16s	remaining: 44.7s
4690:	learn: 0.9273937	total: 11m 16s	remaining: 44.6s
4691:	learn: 0.9273957	total: 11m 16s	remaining: 44.4s
4692:	learn: 0.9273960	total: 11m 16s	remaining: 44.3s
4693:	learn: 0.9273961	total: 11m 17s	remaining: 44.1s
4694:	learn: 0.9273974	total: 11m 17s	remaining: 44s
4695:	learn: 0.9273974	total: 11m 17s	remaining: 43.8s
4696:	learn: 0.9273985	total: 11m 17s	remaining: 43.7s
4697:	learn: 0.9273985	total: 11m 17s	remaining: 43.6s
4698:	learn: 0.9274008	total: 11m 17s	remaining: 43.4s
4699:	learn: 0.9274067	total: 11m 18s	remaining: 43.3s
4700:	learn: 0.9274067	total: 11m 18s	remaining: 43.1s
4701:	learn: 0

4834:	learn: 0.9276166	total: 11m 37s	remaining: 23.8s
4835:	learn: 0.9276167	total: 11m 37s	remaining: 23.6s
4836:	learn: 0.9276167	total: 11m 37s	remaining: 23.5s
4837:	learn: 0.9276170	total: 11m 37s	remaining: 23.4s
4838:	learn: 0.9276170	total: 11m 37s	remaining: 23.2s
4839:	learn: 0.9276172	total: 11m 37s	remaining: 23.1s
4840:	learn: 0.9276177	total: 11m 37s	remaining: 22.9s
4841:	learn: 0.9276191	total: 11m 38s	remaining: 22.8s
4842:	learn: 0.9276198	total: 11m 38s	remaining: 22.6s
4843:	learn: 0.9276198	total: 11m 38s	remaining: 22.5s
4844:	learn: 0.9276198	total: 11m 38s	remaining: 22.3s
4845:	learn: 0.9276215	total: 11m 38s	remaining: 22.2s
4846:	learn: 0.9276222	total: 11m 38s	remaining: 22.1s
4847:	learn: 0.9276286	total: 11m 38s	remaining: 21.9s
4848:	learn: 0.9276286	total: 11m 39s	remaining: 21.8s
4849:	learn: 0.9276286	total: 11m 39s	remaining: 21.6s
4850:	learn: 0.9276286	total: 11m 39s	remaining: 21.5s
4851:	learn: 0.9276298	total: 11m 39s	remaining: 21.3s
4852:	lear

4985:	learn: 0.9277894	total: 11m 58s	remaining: 2.02s
4986:	learn: 0.9277894	total: 11m 59s	remaining: 1.87s
4987:	learn: 0.9277979	total: 11m 59s	remaining: 1.73s
4988:	learn: 0.9277985	total: 11m 59s	remaining: 1.58s
4989:	learn: 0.9278003	total: 11m 59s	remaining: 1.44s
4990:	learn: 0.9278029	total: 11m 59s	remaining: 1.3s
4991:	learn: 0.9278030	total: 11m 59s	remaining: 1.15s
4992:	learn: 0.9278188	total: 11m 59s	remaining: 1.01s
4993:	learn: 0.9278200	total: 12m	remaining: 865ms
4994:	learn: 0.9278202	total: 12m	remaining: 721ms
4995:	learn: 0.9278220	total: 12m	remaining: 577ms
4996:	learn: 0.9278221	total: 12m	remaining: 433ms
4997:	learn: 0.9278222	total: 12m	remaining: 288ms
4998:	learn: 0.9278359	total: 12m	remaining: 144ms
4999:	learn: 0.9278359	total: 12m	remaining: 0us


In [26]:
online_score(pred_catb)

预测结果最大值：17300.406162983807,预测结果最小值：1033.3485877080814
对比913分数:0.969418109128035
